# Bigrams

In [7]:
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
train_dataset = load_dataset("rotten_tomatoes", split="train[:15%]+train[-15%:]")
test_dataset = load_dataset("rotten_tomatoes", split="test[:15%]+test[-15%:]")
train_df = train_dataset.to_pandas()
train_df.sample(frac=1)
test_df = test_dataset.to_pandas()

Found cached dataset rotten_tomatoes (/home/zhenya/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Found cached dataset rotten_tomatoes (/home/zhenya/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


In [17]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=0.8)
X = bigram_vectorizer.fit_transform(train_df["text"])

In [18]:
import numpy as np
train_df["vector"] = train_df["text"].apply(lambda x: bigram_vectorizer.transform([x]).toarray()[0])
test_df["vector"] = test_df["text"].apply(lambda x: bigram_vectorizer.transform([x]).toarray()[0])
X_train = np.stack(train_df["vector"].values, axis=0)
X_test = np.stack(test_df["vector"].values, axis=0)
y_train = train_df["label"].to_numpy()
y_test = test_df["label"].to_numpy()

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
clf = LogisticRegression(C=0.1)
clf = clf.fit(X_train, y_train)

In [20]:
test_df["prediction"] = test_df["vector"].apply(lambda x: clf.predict([x])[0])
print(classification_report(test_df["label"], test_df["prediction"]))

              precision    recall  f1-score   support

           0       0.72      0.75      0.73       160
           1       0.74      0.71      0.72       160

    accuracy                           0.73       320
   macro avg       0.73      0.73      0.73       320
weighted avg       0.73      0.73      0.73       320

